In [ ]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.4 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.4 MB/s eta 0:00:00


### Importing necessary Libraries

In [ ]:
import pandas as pd
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
from torch.utils.data import DataLoader
from transformers import AdamW
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers import default_data_collator
import collections
import random
import csv
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
%env WANDB_DISABLED=True

env: WANDB_DISABLED=True


###Loading the pretrained model

In [ ]:
#Specifying  the name of the pre-trained model to used form hugging face 
Modelname = "deepset/xlm-roberta-base-squad2"

In [ ]:
# cloning the haystack library
!pip install git+https://github.com/deepset-ai/haystack.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-y8ul1ezv
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-y8ul1ezv
  Resolved https://github.com/deepset-ai/haystack.git to commit d322beed6c5a124f1a409eaaa9cdc4621352ec6c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.9/299.9 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.9 MB/s eta 0:00:00
  Prepar

In [ ]:
#installin the FarmReader library which is used by Tokenzier
!pip install FARMReader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement FARMReader (from versions: none)
ERROR: No matching distribution found for FARMReader


### Loading the pretrained model and tokenizer

In [ ]:
# Loading the Pretrained model from about loader
model = AutoModelForQuestionAnswering.from_pretrained(Modelname)
#Loading the Tokenzier for above imported model
tokenizer = AutoTokenizer.from_pretrained(Modelname)

In [ ]:
# helper functions
def set_max_len_encd_stride(max_len, encd_stride):
  # Store the provided values in new variables
    max_len_ = max_len
    encd_stride_ = encd_stride
     # Return the stored values as a tuple
    return max_len_, encd_stride_

## Preprocessing of data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading the tarin file by using pandas 
train_data_chai =  pd.read_csv("/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/train.csv", encoding = "utf-8")
train_data_chai = train_data_chai.sample(frac=1, random_state = random_seed)

In [ ]:

train_data_chai['question'] = train_data_chai['question'].apply(lambda x: x.lstrip())

In [ ]:
train_data_chai= train_data_chai[:].reset_index(drop=True)

valid_data_chai = train_data_chai[-10:].reset_index(drop=True)



### Displaying the few records in tarining data

In [ ]:
train_data_chai.head()

,id,context,question,answer_text,answer_start,language
0,cd1339bdd,एरीजोना (English: Arizona) संयुक्त राज्य अमेरि...,एरिज़ोना की राजधानी क्या है?,फ़िनक्स,123,hindi
1,4c38392a6,"ஆத்திரேலியா, ஆஸ்திரேலியா அல்லது அவுஸ்திரேலியா ...",அவுஸ்திரேலியா நாட்டின் தலைநகரம் எது?,கான்பரா,1472,tamil
2,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi
3,5b2018761,मदर टेरेसा (२६ अगस्त १९१० - ५ सितम्बर १९९७) जि...,मदर टेरेसा की राष्ट्रीयता क्या थी?,भारतीय,328,hindi
4,4f99ce2e0,"எம். ஜி. ஆர் என்ற பெயரில் புகழ் பெற்ற, மருதூர்...",நடிகர் ம. கோ. இராமச்சந்திரன் எப்போது பிறந்தார்?,"சனவரி 17, 1917",99,tamil


In [ ]:
#Loading MLQA file
data_mldq = pd.read_csv("/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/mlqa_hindi (1).csv", encoding = "utf-8")
#Loading data_xquad file
data_xquad = pd.read_csv("/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/xquad (1).csv", encoding = "utf-8")

### Displaying the mldq records

In [ ]:
data_mldq.head()

,context,question,answer_text,answer_start,language
0,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,एक प्रकार की रस्सी का नाम बताईए जो सेलेरी के क...,कोलेन्काइमा,2267,hindi
1,पैरेनकाइमा कोशिकाएं वे जीवित कोशिकाएं हैं जो स...,क्या चीज पूरी तरह से पैरेनकाइमा कोशिकाओं से बन...,पत्तियां,158,hindi
2,जाइलेम कोशिकाएं कोशिका भित्तियों की लिग्निकृत ...,जाइलेम किस प्रकार के पौधों में पाया जाता है?,ट्रेकियोफाइटों,292,hindi
3,फ्लोएम उच्चतर पौधों में आहार का संवहन करने वाल...,लेप्टोम क्या है?\n,समान कार्य करने वाला एक अधिक सरल ऊतक,969,hindi
4,"सभी हवाई अवयवों की बाह्यत्वचा, जड़ों को छोड़कर...",बाह्यत्वचा की कोशिकाएं क्या कर सकती हैं?,क्यटिन का संश्लेषण,433,hindi


In [ ]:
# cleaning the input data by removing leading white space from the beginning of the questions. 
data_mldq['question'] = data_mldq['question'].apply(lambda x: x.lstrip())
data_xquad['question'] = data_xquad['question'].apply(lambda x: x.lstrip())

In [ ]:
#concat() method of Pandas DataFrame is used to concatenate the two input DataFrames vertically (i.e., along rows) by default. 
#This means that the resulting DataFrame will have all the rows of train_data_chai
train_data = pd.concat([train_data_chai, data_mldq])


### Displaying validation data

In [ ]:
valid_data_chai

,id,context,question,answer_text,answer_start,language
0,9aaa1672a,गुरु केलुचरण महापात्र (1926-2004) ने ओड़िसीका ...,गुरु केलुचरण महापात्र का जन्म कब हुआ था?,1926,23,hindi
1,a317a9b97,ஜெ.ஜெ. தாம்சன்(Joseph John Thomson) என்று பொது...,அணுவை முதலில் கண்டுபிடித்தவர் யார்?,சர் ஜோசப் ஜான் தாம்சன்,65,tamil
2,435396d8e,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की परिचालन सीमा कितने किल...,5 हजार किलोमीटर,261,hindi
3,f64641ba1,யானை பாலூட்டி வகையைச் சேர்ந்த ஒரு தாவர உண்ணி (...,ஆண் யானை சராசரி எத்தனை மீட்டர் உயரம் இருக்கும்,3,2284,tamil
4,a52d546ba,பூமியில் வாழும் பாலூட்டிகளில் மிக மிக முன்னேற்...,ஆதி மனிதனின் வேட்டையாட பயன்படுத்திய ஆயுதம் எது?,கற்கருவிகளைப்,2448,tamil
5,3acaa3b86,अरस्तु (384 ईपू – 322 ईपू) यूनानी दार्शनिक थे।...,अरस्तु का जन्म कहाँ हुआ था?,स्टेगेरिया,97,hindi
6,7a3e4bf22,சிந்துவெளி நாகரிகம் (Indus Valley Civilisation...,சிந்து சமவெளி நாகரிகம் எப்போது உருவானது?,கி.மு 3000,305,tamil
7,de34ee0a0,Main Page\n\nआई एन एस (INS) विक्रमादित्य (San...,आई एन एस विक्रमादित्य युद्धपोत किस कंपनी द्वार...,माइकोलैव ब्लैक ‍सी शिपयार्ड,460,hindi
8,518fc3aaf,"अमेरिकी क्रन्तिकारी युद्ध (1775–1783), जिसे सं...",कितने अमेरिकी उपनिवेश अमेरिकी क्रांति का हिस्स...,तेरह,147,hindi
9,632812604,संयुक्त राष्ट्र संगठन का मुख्यालय [[[अमेरिका]]...,संयुक्त राष्ट्र का मुख्यालय कहाँ पर है?,न्युयॉर्क,50,hindi


In [ ]:
#displaying the tamil data
valid_data_chai_tamil = valid_data_chai[valid_data_chai.language == 'tamil']
valid_data_chai_tamil

,id,context,question,answer_text,answer_start,language
1,a317a9b97,ஜெ.ஜெ. தாம்சன்(Joseph John Thomson) என்று பொது...,அணுவை முதலில் கண்டுபிடித்தவர் யார்?,சர் ஜோசப் ஜான் தாம்சன்,65,tamil
3,f64641ba1,யானை பாலூட்டி வகையைச் சேர்ந்த ஒரு தாவர உண்ணி (...,ஆண் யானை சராசரி எத்தனை மீட்டர் உயரம் இருக்கும்,3,2284,tamil
4,a52d546ba,பூமியில் வாழும் பாலூட்டிகளில் மிக மிக முன்னேற்...,ஆதி மனிதனின் வேட்டையாட பயன்படுத்திய ஆயுதம் எது?,கற்கருவிகளைப்,2448,tamil
6,7a3e4bf22,சிந்துவெளி நாகரிகம் (Indus Valley Civilisation...,சிந்து சமவெளி நாகரிகம் எப்போது உருவானது?,கி.மு 3000,305,tamil


In [ ]:
#displaying the hindi data
valid_data_chai_hindi = valid_data_chai[valid_data_chai.language == 'hindi']
valid_data_chai_hindi

,id,context,question,answer_text,answer_start,language
0,9aaa1672a,गुरु केलुचरण महापात्र (1926-2004) ने ओड़िसीका ...,गुरु केलुचरण महापात्र का जन्म कब हुआ था?,1926,23,hindi
2,435396d8e,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की परिचालन सीमा कितने किल...,5 हजार किलोमीटर,261,hindi
5,3acaa3b86,अरस्तु (384 ईपू – 322 ईपू) यूनानी दार्शनिक थे।...,अरस्तु का जन्म कहाँ हुआ था?,स्टेगेरिया,97,hindi
7,de34ee0a0,Main Page\n\nआई एन एस (INS) विक्रमादित्य (San...,आई एन एस विक्रमादित्य युद्धपोत किस कंपनी द्वार...,माइकोलैव ब्लैक ‍सी शिपयार्ड,460,hindi
8,518fc3aaf,"अमेरिकी क्रन्तिकारी युद्ध (1775–1783), जिसे सं...",कितने अमेरिकी उपनिवेश अमेरिकी क्रांति का हिस्स...,तेरह,147,hindi
9,632812604,संयुक्त राष्ट्र संगठन का मुख्यालय [[[अमेरिका]]...,संयुक्त राष्ट्र का मुख्यालय कहाँ पर है?,न्युयॉर्क,50,hindi


### Displaying train data 

In [ ]:
#Displaying train data using head function
train_data.head()

,id,context,question,answer_text,answer_start,language
0,cd1339bdd,एरीजोना (English: Arizona) संयुक्त राज्य अमेरि...,एरिज़ोना की राजधानी क्या है?,फ़िनक्स,123,hindi
1,4c38392a6,"ஆத்திரேலியா, ஆஸ்திரேலியா அல்லது அவுஸ்திரேலியா ...",அவுஸ்திரேலியா நாட்டின் தலைநகரம் எது?,கான்பரா,1472,tamil
2,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi
3,5b2018761,मदर टेरेसा (२६ अगस्त १९१० - ५ सितम्बर १९९७) जि...,मदर टेरेसा की राष्ट्रीयता क्या थी?,भारतीय,328,hindi
4,4f99ce2e0,"எம். ஜி. ஆர் என்ற பெயரில் புகழ் பெற்ற, மருதூர்...",நடிகர் ம. கோ. இராமச்சந்திரன் எப்போது பிறந்தார்?,"சனவரி 17, 1917",99,tamil


In [ ]:
train_data = train_data.reset_index(drop=True)
train_data

,id,context,question,answer_text,answer_start,language
0,cd1339bdd,एरीजोना (English: Arizona) संयुक्त राज्य अमेरि...,एरिज़ोना की राजधानी क्या है?,फ़िनक्स,123,hindi
1,4c38392a6,"ஆத்திரேலியா, ஆஸ்திரேலியா அல்லது அவுஸ்திரேலியா ...",அவுஸ்திரேலியா நாட்டின் தலைநகரம் எது?,கான்பரா,1472,tamil
2,0d35dc007,अग्नि पंचम (अग्नि-५) भारत की अन्तरमहाद्वीपीय ब...,अग्नि पंचम(५) मिसाइल की लम्बाई कितने मीटर है?,17,155,hindi
3,5b2018761,मदर टेरेसा (२६ अगस्त १९१० - ५ सितम्बर १९९७) जि...,मदर टेरेसा की राष्ट्रीयता क्या थी?,भारतीय,328,hindi
4,4f99ce2e0,"எம். ஜி. ஆர் என்ற பெயரில் புகழ் பெற்ற, மருதூர்...",நடிகர் ம. கோ. இராமச்சந்திரன் எப்போது பிறந்தார்?,"சனவரி 17, 1917",99,tamil
...,...,...,...,...,...,...
6534,NaN,"सिडनी शेल्डन (11 फरवरी,1917 - 30 जनवरी 2007) ए...",सिडनी शेल्डन की राष्ट्रीयता क्या थी?,अमेरिकी,48,hindi
6535,NaN,राज्यों को काउंटियों या काउंटी-समकक्ष में विभा...,"अतीत में, सार्वजनिक शिक्षा और सार्वजनिक स्वास्...",राज्य,590,hindi
6536,NaN,"89 वें अकादमी पुरस्कार (ऑस्कर 2017) समारोह, मो...",पुरस्कारों का आयोजन किस दिन किया गया?,"26 फरवरी, 2017",141,hindi
6537,NaN,डीज़ल उत्सर्जन तरल (अंग्रेजी:Diesel exhaust fl...,डीईएफ का क्या अर्थ होता है?,(अंग्रेजी:Diesel exhaust fluid,19,hindi


In [ ]:
# train_data = train_data.sample(frac=1, random_state = random_seed)

In [ ]:
#Preparation of data for training the model
train_dataset = Dataset.from_pandas(train_data)
valid_dataset = Dataset.from_pandas(valid_data_chai)
train_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 6539
})

In [ ]:
#  set_max_len_encd_stride() function is used to set the maximum length of input sequences to 384 and the encoding stride to 128
max_len, encoding_stride = set_max_len_encd_stride(384, 128)
print(max_len)
print(encoding_stride)


384
128


In [ ]:
#demonstrates how to use the tokenizer object to preprocess input data for model 
#including how to customize the tokenization process by setting various parameters.
train_encoding = tokenizer(train_dataset['question'], train_dataset['context'],
                             truncation = 'only_second', max_length=max_len, stride = encoding_stride, 
                             return_overflowing_tokens = True, return_offsets_mapping = True,
                             padding = 'max_length')
valid_encoding = tokenizer(valid_dataset['question'], valid_dataset['context'],
                             truncation = 'only_second', max_length=max_len, stride = encoding_stride, 
                             return_overflowing_tokens = True, return_offsets_mapping = True,
                             padding = 'max_length')

In [ ]:
class proc_data():
    #handle_no_answer checks if there is no answer and if it's the case, it will append the start and end positions 
    def handle_no_answer(self, data_encoding, answer_start, cls_idx):
        if len(answer_start) == 0:
            data_encoding['start_positions'].append(cls_idx)
            data_encoding['end_positions'].append(cls_idx)
            return True
        return False
    # method correct_tokens receives the sequence ids and returns the index of the first and last token corresponding to the answer.
    def correct_tokens(self, seq_ids):
        seq_ids = list(seq_ids)
        answer_token_start_index = seq_ids.index(1)
        seq_ids_reverse = seq_ids[::-1]
        answer_token_end_index_reverse = seq_ids_reverse.index(1)
        answer_token_end_index = len(seq_ids) - 1 - answer_token_end_index_reverse
        return answer_token_start_index, answer_token_end_index
# method update_data_encoding appends the start and end positions of the answer token to the data encoding dictionary.    
    def update_data_encoding(self, data_encoding, start, end):
        data_encoding['start_positions'].append(start)
        data_encoding['end_positions'].append(end)
        #method get_encodings receives the data encoding, data sample, tokenizer, offset mapping, and sample mapping
    def get_encodings(self, data_encoding, data_sample, tokenizer, offset_mapping, sample_mapping):
        for i, offsets in enumerate(offset_mapping):
            input_ids = data_encoding['input_ids'][i]
            seq_ids = data_encoding.sequence_ids(i)
            sample_index = sample_mapping[i]
            answer_start = [data_sample['answer_start'][sample_index]]
            text = data_sample['answer_text'][sample_index]
# handling no answe function
            no_answer_handled = self.handle_no_answer(data_encoding, answer_start, input_ids.index(tokenizer.cls_token_id))
            if not no_answer_handled:
                answer_start_char_index = answer_start[0]
                answer_end_char_index = answer_start_char_index + len(text)

                answer_token_start_index, answer_token_end_index = self.correct_tokens(seq_ids)

                if offsets[answer_token_start_index][0] <= answer_start_char_index and offsets[answer_token_end_index][1] >= answer_end_char_index:
                    while offsets[answer_token_end_index][1] >= answer_end_char_index:
                        answer_token_end_index -=1
                    while answer_token_start_index <len(offsets) and offsets[answer_token_start_index][0] <= answer_start_char_index:
                        answer_token_start_index +=1
                        
                    self.update_data_encoding(data_encoding, answer_token_start_index-1, answer_token_end_index+1)
                else:
                    self.handle_no_answer(data_encoding, [], input_ids.index(tokenizer.cls_token_id)) # [] is just a dummpy,  a 0 element list for the function to work correctly. change this afterwards
        return data_encoding, offset_mapping, sample_mapping
    # function that adds start and token positions
    def add_start_end_token_positions(self, data_encoding, data_sample, tokenizer, train = True):
        offset_mapping = data_encoding.pop('offset_mapping')
        sample_mapping = data_encoding.pop("overflow_to_sample_mapping")
        if train:
            data_encoding['start_positions'] = []
            data_encoding['end_positions'] = []

            data_encoding, offset_mapping, sample_mapping = self.get_encodings(data_encoding, data_sample, tokenizer, offset_mapping, sample_mapping)

        return data_encoding, offset_mapping, sample_mapping

In [ ]:
 #instantiates an object of the proc_data class and assigns it to the variable proc_data_obj
proc_data_obj = proc_data()

In [ ]:
# Traing encd_proc
train_encd_proc, train_offset_mapping, training_sample_mapping = proc_data_obj.add_start_end_token_positions(train_encoding, train_data, tokenizer, True)


In [ ]:
#valid_encd_proc
valid_encd_proc, valid_offset_mapping, valid_sample_mapping = proc_data_obj.add_start_end_token_positions(valid_encoding, valid_data_chai, tokenizer, True)

In [ ]:
#printing the start positions count
print(len(train_encd_proc['start_positions']))
#printing the end positions count
print(len(train_encd_proc['end_positions']))
#printing the input_ids count
print(len(train_encd_proc['input_ids']))
# print(len(train_encd_proc['attention_masks']))


21727
21727
21727


In [ ]:
# creating chai datatset class wjich takes encoding and id and returns len
class ChaiiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
# training dataset
train_dataset_ = ChaiiDataset(train_encd_proc)
#validation dataset
valid_dataset_ = ChaiiDataset(valid_encd_proc)

In [ ]:
pip install git+https://github.com/huggingface/accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-abqq7lyc
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-abqq7lyc
  Resolved https://github.com/huggingface/accelerate to commit ab379793d44be16d8fcac5c098a3ab9b6f5a7ec3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.20.0.dev0-py3-none-any.whl size=222939 sha256=79dbcca23c597babd05928bc561784426c001d3279193141de563234b7923e93
  Stored in directory: /tmp/pip-ephem-wheel-cache-oipzalgu/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate


In [ ]:
# train args passing args
train_args = TrainingArguments(
    output_dir='./results',          
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,  
    learning_rate = 3e-5, # default 5e-5
    gradient_accumulation_steps = 8,
    warmup_steps=0.1,
    weight_decay=0.01,
    num_train_epochs = 1,

  # logging_dir=logs            
#     logging_steps=100,
    save_strategy = 'epoch',
    evaluation_strategy = 'epoch',
#     eval_steps = 100,
    save_total_limit = 1,
    load_best_model_at_end = True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


### loading the trainer class

In [ ]:
trainer = Trainer(
    model=model,                         
    args=train_args,                  
    train_dataset=train_dataset_,         
    eval_dataset=valid_dataset_,
    data_collator = default_data_collator
)

### Training the model

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.824500,0.147810


TrainOutput(global_step=679, training_loss=0.7988492940416744, metrics={'train_runtime': 1739.3245, 'train_samples_per_second': 12.492, 'train_steps_per_second': 0.39, 'total_flos': 4257895975202304.0, 'train_loss': 0.7988492940416744, 'epoch': 1.0})

## Testing the data

In [ ]:
test_dataanswers=['22bf73dec',
'282758170',
'd60987e0e',
'f99c770dc',
'40dec1964']

In [ ]:
test_data =  pd.read_csv("/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/test.csv", encoding = "utf-8")
# test_data['context'] = test_data['context'].apply(lambda x: x.lstrip())
test_data['question'] = test_data['question'].apply(lambda x: x.lstrip())

test_dataset = Dataset.from_pandas(test_data)

test_encoding = tokenizer(test_dataset['question'], test_dataset['context'],
                             truncation = 'only_second', max_length=max_len, stride = encoding_stride, 
                             return_overflowing_tokens = True, return_offsets_mapping = True,
                             padding = 'max_length')

test_encd_proc, test_offset_mapping,test_sample_mapping = proc_data_obj.add_start_end_token_positions(test_encoding, test_data, tokenizer, False)
test_dataset_encd = ChaiiDataset(test_encd_proc)

In [ ]:
test_data

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [ ]:
test_encd_proc.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
test_encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
predictions = trainer.predict(test_dataset_encd)

## Validset prediction

In [ ]:
valid_encoding_ = tokenizer(valid_dataset['question'], valid_dataset['context'],
                             truncation = 'only_second', max_length=max_len, stride = encoding_stride, 
                             return_overflowing_tokens = True, return_offsets_mapping = True,
                             padding = 'max_length')

In [ ]:
valid_encoding_.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
valid_encd_proc_, valid_offset_mapping_,valid_sample_mapping_ = proc_data_obj.add_start_end_token_positions(valid_encoding_, valid_dataset, tokenizer, False)
# valid_encd_proc_.pop('id')
valid_dataset_encd = ChaiiDataset(valid_encd_proc_)

In [ ]:
valid_encoding_.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
valid_predictions = trainer.predict(valid_dataset_encd)

In [ ]:
#
def get_default_corr_ans_dict():
    return {"text": "", "score": 0.0}

def get_logits_mapping(start_logits, end_logits, offset_mapping, index, check_best):
    start_logit, end_logit, offsets = np.argsort(start_logits[index]), np.argsort(end_logits[index]), offset_mapping[index]
    
    return start_logit[-1 : -check_best - 1 : -1].tolist(), end_logit[-1 : -check_best - 1 : -1].tolist(), offsets, start_logits[index], end_logits[index]

def post_processing(data, data_encoding, st_logits, en_logits , sample_mapping, offset_mapping, answer_length_limit = 30, check_best = 20):
    start_logits = st_logits
    end_logits = en_logits
    mapping_data_index_sample = collections.defaultdict(list) # of the form [[1,2,3,4,], [5,6,7,8]]        
    for sample_idx, data_idx in enumerate(sample_mapping):
        mapping_data_index_sample[data_idx].append(sample_idx)
    
    predicted_answers = collections.OrderedDict()
    for data_index, data_point in enumerate(data):
        encoding_idxs = mapping_data_index_sample[data_index]
        context = data_point['context']
        answers = {}
        for index in encoding_idxs:
            assert True
            selected_start_logits_index, selected_end_logits_index, offsets, start_logit, end_logit = get_logits_mapping(start_logits, end_logits, offset_mapping, index, check_best)
            
            seq_ids = data_encoding.sequence_ids(index)
            for start in selected_start_logits_index:
                if(start >= len(offsets) or seq_ids[start] != 1):
                    continue
                for end in selected_end_logits_index:
                    if(end >= len(offsets) or seq_ids[end]!= 1 or end < start or (end-start) + 1> answer_length_limit):
                        continue
                    key = start_logit[start] + end_logit[end]
                    answers[key] = context[offsets[start][0]: offsets[end][1]]
                    
        scores = list(answers.keys())
        if len(scores) > 0:
            sorted_scores = sorted(scores)
            best_score = sorted_scores[-1]
            best_answer = answers[best_score]

        else:
            best_answer = ""

        predicted_answers[data_point["id"]] = best_answer
        
    return predicted_answers

In [ ]:
final_predictions_valid = post_processing(valid_dataset, valid_encoding_, valid_predictions.predictions[0], valid_predictions.predictions[1], valid_sample_mapping_, valid_offset_mapping_)

In [ ]:
valid_dataset

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language'],
    num_rows: 10
})

## Evaluation

In [ ]:
final_predictions = post_processing(test_dataset, test_encoding, predictions.predictions[0], predictions.predictions[1], test_sample_mapping, test_offset_mapping)

In [ ]:
key = final_predictions.keys()

In [ ]:
for k in key:
    print((k, final_predictions[k]))

('22bff3dec', ' येलन')
('282758170', '\nदिसंबर 2005')
('d60987e0e', '१२ मार्च १८२४')
('f99c770dc', ' 13')
('40dec1964', 'சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்')


In [ ]:
sample = pd.read_csv('/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/sample_submission.csv')
sample.columns

Index(['id', 'PredictionString'], dtype='object')

In [ ]:

submission = [[str(k), str(final_predictions[k])] for k in key]
with open("submission.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(submission)

sub_df = pd.read_csv('submission.csv', header = None)
sub_df = sub_df.rename(columns={0: 'id', 1: 'PredictionString'})
print(sub_df)
sub_df.to_csv('submission.csv', index=None)

          id                     PredictionString
0  22bff3dec                                 येलन
1  282758170                        \nदिसंबर 2005
2  d60987e0e                        १२ मार्च १८२४
3  f99c770dc                                   13
4  40dec1964  சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


In [ ]:
!cat submission.csv

id,PredictionString
22bff3dec, येलन
282758170,"
दिसंबर 2005"
d60987e0e,१२ मार्च १८२४
f99c770dc, 13
40dec1964,சுவாமிநாதன் மற்றும் வர்கீஸ் குரியன்


In [ ]:
#Evaluation metrics

In [ ]:
def jaccard(row): 
    str1 = row[0]
    str2 = row[1]
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

### F1 score

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

# Convert the ground truth and predicted answers into lists
ground_truth = test_dataanswers[:]
 

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/nlp final project/chaii-hindi-and-tamil-question-answering/sample_submission.csv')

# Retrieve the first column as a list
predicted_answers = df.iloc[:, 0].tolist()

# Compute the F1 score
f1 = f1_score(ground_truth, predicted_answers, average='micro')
print("F1 Score %:", f1*100)

F1 Score %: 80.00000000000001
